In [5]:
import csv
import json
import os

# Define paths
data_directory = 'data-ref'
csv_filename = 'nasdaq.csv'  # Using the actual filename from your output
json_filename = 'nasdaq.json'

# Full path to the CSV file
csv_filepath = os.path.join(data_directory, csv_filename)

# Read CSV into a list of strings (not lists)
data_list = []
try:
    with open(csv_filepath, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            # Take the first element of each row, assuming it's the ticker
            if row:  # Make sure the row isn't empty
                data_list.append(row[0])  # Add just the string, not a list
    
    # Create the JSON structure
    json_data = {"data": data_list}
    
    # Write to JSON file
    with open(json_filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(json_data, jsonfile)
    
    print(f"Successfully converted {csv_filepath} to {json_filename}")
    
    # Preview the data
    print("\nPreview of the data:")
    print(json_data)
    
except FileNotFoundError:
    print(f"Error: The file {csv_filepath} was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Successfully converted data-ref/nasdaq.csv to nasdaq.json

Preview of the data:
{'data': ['ATVI', 'ADBE', 'ADP', 'ABNB', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AMD', 'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'AZN', 'TEAM', 'ADSK', 'BKR', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CHTR', 'CTAS', 'CSCO', 'CTSH', 'CMCSA', 'CEG', 'CPRT', 'CSGP', 'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FANG', 'DLTR', 'EBAY', 'EA', 'ENPH', 'EXC', 'FAST', 'FISV', 'FTNT', 'GILD', 'GFS', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'ISRG', 'JD', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LCID', 'LULU', 'MAR', 'MRVL', 'MELI', 'META', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MNST', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'ODFL', 'PCAR', 'PANW', 'PAYX', 'PYPL', 'PDD', 'PEP', 'QCOM', 'REGN', 'RIVN', 'ROST', 'SGEN', 'SIRI', 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZM', 'ZS']}
